**9.- Diseña un modelo en Keras que utilice capas convolucionales 2D para predecir los valores de cierre Apple utilizando como entrada imágenes de la evolución del valor durante los 4 días anteriores. Usa el tipo de imágenes que creas mas conveniente (de barras, de velas, lineal…).**


In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Layer, Dense, GRU, Conv2D, Flatten, Dropout, BatchNormalization, MaxPooling2D, LSTM
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib import image
import tensorflow as tf
from PIL import Image

In [2]:
#Leemos los datos de apple
apple = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\aa.us.txt", delimiter=",")
#Me quedo con los datos de apertura
open_data = apple['Open'].values
open_data = open_data[:1376]
close_data = apple['Close'].values[:1376]

In [3]:
#Generamos un array de ceros
LAG = 5
data = np.zeros((open_data.shape[0]- LAG, LAG))
close = np.zeros((close_data.shape[0]- LAG, LAG))

In [4]:
#Rellenamos los arrays con los datos correspondientes
for i in range(0,(open_data.shape[0] - LAG)):
    data[i,:] = open_data[i:i+LAG]
    close[i,:] = close_data[i:i+LAG]

In [5]:
#Establecemos las labels de los valores de cierre de apple
y_train = close[:round(data.shape[0]*0.7),-1]
y_test = close[round(data.shape[0]*0.7):,-1]

In [6]:
y_train.shape

(960,)

In [13]:
imagenes = []
for i in range(0,data.shape[0]):
    fig = plt.figure(figsize=(2, 2))
    plt.plot(data[i,:])
    fig.canvas.draw()
    arrayRGB = np.array(fig.canvas.renderer._renderer)
    #Convertimos las imagenes a blanco y negro
    imagenes.append(np.dot(arrayRGB[...,:3], [0.2989, 0.5870, 0.1140]))
    plt.clf()
    plt.close
    

<ipython-input-13-6c7f2c73426d>:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(2, 2))


<Figure size 144x144 with 0 Axes>

<Figure size 144x144 with 0 Axes>

<Figure size 144x144 with 0 Axes>

<Figure size 144x144 with 0 Axes>

...

<Figure size 144x144 with 0 Axes>

<Figure size 144x144 with 0 Axes>

<Figure size 144x144 with 0 Axes>

<Figure size 144x144 with 0 Axes>

<Figure size 144x144 with 0 Axes>

In [19]:
len(imagenes[0])

144

In [16]:
n_ejemplos = data.shape[0]
data = np.zeros((n_ejemplos, 144,144))
data.shape

(1371, 144, 144)

In [17]:
#Metemos las imagenes en nuestro array de datos.
for i in range(0,data.shape[0]):
    data[i,:,:] = imagenes[i]

In [12]:
x_train = data[:round(data.shape[0]*0.7),:,:]
x_test = data[round(data.shape[0]*0.7):,:,:]

In [14]:
#Como hemos obtenido las imagenes en blancos y negros, las reorganizamos para añadirle el canal 1 para las conv2D
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

In [16]:
x_train.shape

(960, 144, 144, 1)

In [17]:
model = Sequential()
model.add(Conv2D(50, 3, input_shape=(144,144,1), padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2))
model.add(Conv2D(30, 3, padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(1))

# mostramos el modelo
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 144, 144, 50)      500       
                                                                 
 dropout (Dropout)           (None, 144, 144, 50)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 72, 72, 50)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 30)        13530     
                                                                 
 dropout_1 (Dropout)         (None, 72, 72, 30)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 30)       0         
 2D)                                                  

In [18]:
model.compile(optimizer='adam',loss='mae')

In [19]:
history = model.fit(x_train, y_train,  epochs=20)

Epoch 1/20
30/30 [==============================] - 16s 538ms/step - loss: 196.7188
Epoch 2/20
30/30 [==============================] - 16s 542ms/step - loss: 22.3083
Epoch 3/20
30/30 [==============================] - 16s 544ms/step - loss: 14.1727
Epoch 4/20
30/30 [==============================] - 16s 548ms/step - loss: 13.3011
Epoch 5/20
30/30 [==============================] - 16s 546ms/step - loss: 11.4660
Epoch 6/20
30/30 [==============================] - 16s 541ms/step - loss: 10.5362
Epoch 7/20
30/30 [==============================] - 17s 580ms/step - loss: 9.5791
Epoch 8/20
30/30 [==============================] - 18s 588ms/step - loss: 8.1808
Epoch 9/20
30/30 [==============================] - 17s 562ms/step - loss: 6.9801
Epoch 10/20
30/30 [==============================] - 17s 570ms/step - loss: 6.4177
Epoch 11/20
30/30 [==============================] - 17s 559ms/step - loss: 6.0587
Epoch 12/20
30/30 [==============================] - 17s 557ms/step - loss: 5.4776
Epoch 

In [20]:
model.evaluate(x_test, y_test)

13/13 [==============================] - 1s 76ms/step - loss: 2.2091


2.2091047763824463